In [1]:
using Flux
using Flux.Data: DataLoader
using PyCall
using TOML
using Pkg
using Random
using Plots
using PlotThemes
using BSON: @save

In [2]:
env_path = TOML.parse(read("mltools.toml", String))["env_path"]
Pkg.activate(env_path)

  Activating environment at `C:\Users\nikhi\OneDrive\Documents\julia\MLTools\Project.toml`


In [3]:
import MLTools as mlt

In [4]:
df = mlt.getdf(joinpath(@__DIR__, "filtered_dataset.csv"))

,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=C(C=C3)B...,Active
1,COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=CC=C3)N)...,Active
2,C1CN(CCN1)C(=O)C2=C(SC(=C2)C3=CC=CC=C3)N,Active
3,C1=CC=C(C=C1)C(=O)C2=C(C(=C(N2)N)C(=O)N)C3=CC=...,Active
4,CC1=C(C(=O)OC(C1)[C@@H](C)C2CC[C@@H]3[C@@]2(CC...,Active
...,...,...
18229,C1CN(CCC1CNC(=O)NC2=CC=CC=C2)C3=CC=C(C=C3)S(=O...,Inactive
18230,CN1C(=O)C(=C(N(C1=O)CC2=CC=CC=C2)N)C(=O)CN3CCOCC3,Inactive
18231,COC1=C(C=C(C=C1)Cl)NC(=S)NCCC2=CC=C(C=C2)S(=O)...,Inactive
18232,C1CN(CCN1C2=CC=C(C=C2)[N+](=O)[O-])C(=O)CN3C(=...,Inactive


In [5]:
smiles = mlt.dfToStringMatrix(df)

18234×2 Matrix{String}:
 "COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=C(C=C3)Br)N)C(=O)C4=CC=CC=C4)OC"                                                         …  "Active"
 "COC1=C(C=C(C=C1)C2=C(NC(=C2C(=O)C3=CC=CC=C3)N)C(=O)C4=CC=CC=C4)OC"                                                                "Active"
 "C1CN(CCN1)C(=O)C2=C(SC(=C2)C3=CC=CC=C3)N"                                                                                         "Active"
 "C1=CC=C(C=C1)C(=O)C2=C(C(=C(N2)N)C(=O)N)C3=CC=CC=C3Br"                                                                            "Active"
 "CC1=C(C(=O)OC(C1)[C@@H](C)C2CC[C@@H]3[C@@]2(CCC4C3C[C@@H]5[C@]6([C@@]4(C(=O)C=C[C@@H]6O)C)O5)C)CO"                                "Active"
 "CC1CCC/C=C/C(=O)C2=C(CC(=O)O1)C=C(C=C2O)O"                                                                                     …  "Active"
 "CC1C2C(NC(=O)C23C(/C=C(/CCC(C(/C=C/C3=O)O)O)\\C)C=C1C)CC(C)C"                                                                   

In [6]:
vocab = mlt.dfToStringMatrix(mlt.getdf(joinpath(@__DIR__, "vocab.csv")))

71×1 Matrix{String}:
 "#"
 "("
 ")"
 "."
 "/"
 "1"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 ⋮
 "[Nd+3]"
 "[O-]"
 "[OH-]"
 "[P+]"
 "[Pt+2]"
 "[Pt]"
 "[S+]"
 "[S-]"
 "[Se]"
 "[Si]"
 "[Zn+2]"
 "\\"

In [7]:
tokenizer = Dict(j => i for (i, j) in enumerate(vocab))
reverse_tokenizer = Dict(value => key for (key, value) in tokenizer)

Dict{Int64, String} with 71 entries:
  5  => "/"
  56 => "[NH3+]"
  16 => "B"
  35 => "[C@H]"
  55 => "[NH2+]"
  20 => "F"
  60 => "[Nd+3]"
  30 => "[Ba+2]"
  19 => "Cl"
  32 => "[C-]"
  49 => "[N+]"
  6  => "1"
  67 => "[S-]"
  45 => "[I-]"
  44 => "[Hg]"
  9  => "4"
  31 => "[Br-]"
  64 => "[Pt+2]"
  61 => "[O-]"
  29 => "[B-]"
  46 => "[K+]"
  57 => "[NH4+]"
  70 => "[Zn+2]"
  4  => "."
  13 => "8"
  ⋮  => ⋮

In [8]:
function readPyFile(path)
    py"""
    exec(open($(path)).read(), globals(), locals())
    """
end

readPyFile("tokenization.py")
return_tokens(str) = py"return_tokens"(str)

return_tokens (generic function with 1 method)

In [9]:
strings = [[tokenizer[j] for j in return_tokens(i)[begin]] for i in smiles[:, begin]]
activity = reduce(hcat, [i == "Active" ? [1, 0] : [0, 1] for i in smiles[:, end]])'

@assert length(strings) == size(activity)[begin]

In [10]:
# println("average len is $(sum(length.(strings))/length(strings))"

# test_tokens = [reverse_tokenizer[i] for i in strings[begin]]
# test_string = join(test_tokens)

# println("test reverse tokenization: $test_string")

# println("max length string: $(join([reverse_tokenizer[i] for i in strings[findmax(length.(strings))[end]]])))")

In [11]:
convert_back(x) = join([i in keys(reverse_tokenizer) ? reverse_tokenizer[i] : "" for i in x])

convert_back (generic function with 1 method)

In [12]:
max_length = maximum(length.(strings))

190

[https://fluxml.ai/Flux.jl/v0.2/examples/char-rnn.html]

[https://emoryraphael.medium.com/basic-sentiment-analysis-with-julia-using-lstm-e12d4754ee6b]

could potentially use smiles enumeration for data augmentation during training

In [13]:
function pad_features(input_strings, length_max)
    features = []
    for i in input_strings
        dim = size(i)[1]
        pad_size = length_max - dim 
        if pad_size > 0
            pad_array = zeros(Int64, pad_size)
            result = append!(pad_array, i)
        else
            result = i[1:length_max]
        end
        push!(features, result)
    end
    return features
end

pad_features (generic function with 1 method)

In [14]:
# sample_padded = pad_features(strings, max_length)[begin]
# convert_back(sample_padded)

In [15]:
# return a json of relevant model metadata

In [16]:
padded_features = pad_features(strings, max_length)

18234-element Vector{Any}:
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  18, 15, 18, 18, 15, 18, 9, 3, 23, 18]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  18, 15, 18, 18, 15, 18, 9, 3, 23, 18]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  18, 18, 15, 18, 18, 15, 18, 8, 3, 22]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  15, 18, 18, 15, 18, 18, 15, 18, 8, 17]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  3, 18, 3, 23, 10, 3, 18, 3, 18, 23]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  15, 18, 2, 18, 15, 18, 7, 23, 3, 23]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  18, 6, 18, 3, 18, 18, 2, 18, 3, 18]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  3, 23, 3, 23, 3, 23, 3, 23, 3, 23]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  18, 18, 15, 18, 11, 10, 3, 18, 3, 18]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  8, 3, 23, 3, 23, 18, 3, 18, 3, 18]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  18, 8, 3, 23, 18, 3, 23, 18, 3, 18]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  18, 9, 15, 22, 25, 22, 15, 18, 9, 8]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  19, 3, 49, 2, 15, 23, 3, 61, 3, 23]
 ⋮
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  2

In [17]:
function onehot(x)
    onehot_array = zeros(length(keys(tokenizer)) + 1)
    onehot_array[x + 1] = 1
    return onehot_array
end

onehot (generic function with 1 method)

In [18]:
parsed = [onehot.(i) for i in padded_features]
println(length(parsed))

18234


In [19]:
matrix_parsed = [mapreduce(permutedims, vcat, i) for i in parsed]

18234-element Vector{Matrix{Float64}}:
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0

In [20]:
X = [matrix_parsed[i] for i in 1:length(strings)]
Y = Vector{eltype(activity)}[eachrow(activity)...]

18234-element Vector{Vector{Int64}}:
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 [1, 0]
 ⋮
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]
 [0, 1]

In [21]:
trainX, trainY, testX, testY = mlt.trainTestSplit(X, Y, percent_train=.9)
size(trainX), size(trainY), size(testX), size(testY)

((16410, 1), (16410, 1), (1824, 1), (1824, 1))

In [22]:
trainX = convert.(Matrix{Float32}, trainX)
trainY = convert.(Matrix{Float32}, trainY)
testX = convert.(Matrix{Float32}, testX)
testY = convert.(Matrix{Float32}, testY)

1824×1 Matrix{Matrix{Float32}}:
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 ⋮
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]

In [26]:
batch(arr, n) = [arr[i:min(i + n - 1, end)] for i in 1:n:length(arr)]

batch (generic function with 1 method)

In [ ]:
trainX = batch(trainX, 32)
trainY = batch(trainY, 32)
testX = batch(testX, 32)
testY = batch(testY, 32)

In [30]:
N = length(keys(tokenizer)) + 1

model = Chain(
    LSTM(N => 128), relu,
    LSTM(128 => 16), relu,
    Dense(16 => 2), softmax
)

Chain(
  Recur(
    LSTMCell(72 => 128),                # 103_168 parameters
  ),
  NNlib.relu,
  Recur(
    LSTMCell(128 => 16),                # 9_312 parameters
  ),
  NNlib.relu,
  Dense(16 => 2),                       # 34 parameters
  NNlib.softmax,
)         # Total: 12 trainable arrays, 112_514 parameters,
          # plus 4 non-trainable, 288 parameters, summarysize 440.305 KiB.

In [31]:
function eval_model(x)    
    out = model(x)[:, end]
    Flux.reset!(model)

    return out'
end

function loss(x, y)
    return Flux.crossentropy(eval_model(x), y)
end

loss (generic function with 1 method)

In [ ]:
# model(convert(Matrix{Float32}, trainX[begin])[:, 1])
# eval_model(convert(Matrix{Float32}, trainX[begin]))
# eval_model(convert(Matrix{Float32}, trainX[begin]))
# loss(convert(Matrix{Float32}, trainX[begin]), convert(Matrix{Float32}, trainY[begin]))

In [36]:
function accuracy(x, y)
    # check accuracy of prediction
    # ŷ
    ŷ = eval_model(x)
    return findmax(ŷ) == findmax(y)
end

accuracy (generic function with 1 method)

In [ ]:
# https://spcman.github.io/getting-to-know-julia/deep-learning/vision/flux-cnn-zoo/
# maybe just directly input?

# losses = []

# for i in 1:length(trainX)
#     push!(losses, loss(convert(Matrix{Float32}, trainX[i]), convert(Matrix{Float32}, trainY[i])))
# end

In [ ]:
data = [i for i in zip(trainX, trainY)]
train_data_len = length(data)
test_data_len = length(testX)

In [38]:
# data = DataLoader((convert(Matrix{Float32}, trainX), convert(Matrix{Float32}, trainY)); batchsize=32, shuffle=true)
opt = Adam(.001)
epochs = 1:100

training_losses = []
testing_losses = []
training_accs = []
testing_accs = []
preds = Matrix{Float32}(undef, 0, size(X)[begin])

for i in epochs
    println("Epoch: $i")
    Flux.train!(loss, Flux.params(model), data, opt)

    push!(training_losses, loss(trainX, trainY))
    push!(testing_losses, loss(testX, testY))

    push!(training_accs, sum(accuracy.(trainX, trainY)) / train_data_len)
    push!(testing_accs, sum(accuracy.(testX, testY)) / test_data_len)

    println("Training Accuracy: $(training_accs[i])")
    println("Testing Accuracy: $(testing_accs[i])")

    println("Training Loss: $(training_losses[i])")
    println("Testing Loss: $(testing_losses[i])")

    # shuffle!(data)
end